# HousePrices DataSet Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
train1Df = pd.read_excel('HousePrice-Train.xlsx')
trainDf = train1Df.drop('SalePrice', axis=1)

testDf = pd.read_excel('HousePrices-Test.xlsx')

birlesik_df = pd.concat([trainDf, testDf], ignore_index=True)

#

# MSSubClass - MSZoning - Street - Alley - LotShape - LandCountour/LandSlope - Utilities - LotConfig

In [3]:
MSSubClassDf = birlesik_df[['MSSubClass']].copy()
MSSubClassDfEncoded = pd.get_dummies(MSSubClassDf, columns=['MSSubClass'], prefix='MSSubClass')
df1 = birlesik_df.drop('MSSubClass', axis=1)
df1 = pd.concat([df1.iloc[:, :1], MSSubClassDfEncoded, df1.iloc[:, 1:]], axis=1)
#####
MSZoningDf = pd.DataFrame(df1["MSZoning"])
MSZoningDfEncoded = pd.get_dummies(MSZoningDf, columns=['MSZoning'], prefix='MSZoning')
df1 = df1.drop('MSZoning', axis=1)
df1 = pd.concat([df1.iloc[:, :17], MSZoningDfEncoded, df1.iloc[:, 17:]], axis=1)
#####
df1['StreetPave']=df1['Street'].replace({'Pave': 1, 'Grvl':0}).astype(bool)
df1.insert(25, 'StreetPave', df1.pop('StreetPave'))
df1 = df1.drop('Street',axis=1)
#####
df1['Alley'].fillna('None', inplace=True)
AlleyDf = pd.DataFrame(df1["Alley"])
AlleyDfEncoded = pd.get_dummies(AlleyDf, columns=['Alley'], prefix='Alley')
df1 = df1.drop('Alley', axis=1)
df1 = pd.concat([df1.iloc[:, :25], AlleyDfEncoded, df1.iloc[:, 25:]], axis=1)
#####
df1['LotShape']=df1['LotShape'].replace({'Reg': 1, 'IR1': 1, 'IR2': 0, 'IR3': 0}).astype(bool)
#####
LandlerDf = df1[['LandContour','LandSlope']].copy()
lands = [
    ('Bnk', 'Gtl', 'BnkGtl'), ('Bnk', 'Mod', 'BnkMod'), ('Bnk', 'Sev', 'BnkSev'),
    ('HLS', 'Gtl', 'HLSGtl'), ('HLS' , 'Mod', 'HLSMod'), ('HLS', 'Sev', 'HLSSev'),
    ('Low', 'Gtl', 'LowGtl'), ('Low', 'Mod', 'LowMod'), ('Low', 'Sev', 'LowSev'),
    ('Lvl', 'Mod', 'LvlMod'), ('Lvl', 'Sev', 'LvlSev'), ('Gtl', 'Lvl', 'GtlLvl')
]
for land in lands:
    LandlerDf[land[2]] = np.where((LandlerDf['LandSlope'] == land[0]) & (LandlerDf['LandContour'] == land[1]), True, False)
LandlerSonDf = LandlerDf.drop(['LandContour', 'LandSlope'], axis=1)
df1 = df1.drop(['LandContour', 'LandSlope'], axis=1)
df1 = pd.concat([df1.iloc[:, :29], LandlerSonDf, df1.iloc[:, 29:]], axis=1)
#####
df1['Utilities'].fillna('None', inplace=True)
UtilitiesDf = pd.DataFrame(df1['Utilities'])
UtilitiesDf['Utilities_Electric'] = ''
UtilitiesDf['Utilities_Water'] = ''
UtilitiesDf['Utilities_Gas'] = ''
UtilitiesDf['Utilities_Sewage'] = ''
UtilitiesDf['Utilities_None']= ''
if 'AllPub' in UtilitiesDf['Utilities'].values:
    UtilitiesDf['Utilities_Electric'] = True
    UtilitiesDf['Utilities_Water'] = True
    UtilitiesDf['Utilities_Gas'] = True
    UtilitiesDf['Utilities_Sewage'] = True
    UtilitiesDf['Utilities_None'] = False
elif 'NoSewr' in UtilitiesDf['Utilities'].values:
    UtilitiesDf['Utilities_Electric'] = True
    UtilitiesDf['Utilities_Water'] = True
    UtilitiesDf['Utilities_Gas'] = True
    UtilitiesDf['Utilities_Sewage'] = False
    UtilitiesDf['Utilities_None'] = False
elif 'NoSeWa' in UtilitiesDf['Utilities'].values:
    UtilitiesDf['Utilities_Electric'] = True
    UtilitiesDf['Utilities_Water'] = False
    UtilitiesDf['Utilities_Gas'] = True
    UtilitiesDf['Utilities_Sewage'] = False
    UtilitiesDf['Utilities_None'] = False
elif 'ELO' in UtilitiesDf['Utilities'].values:
    UtilitiesDf['Utilities_Electric'] = True
    UtilitiesDf['Utilities_Water'] = False
    UtilitiesDf['Utilities_Gas'] = False
    UtilitiesDf['Utilities_Sewage'] = False
    UtilitiesDf['Utilities_None'] = False
elif 'None' in UtilitiesDf['Utilities'].values:
    UtilitiesDf['Utilities_Electric'] = False
    UtilitiesDf['Utilities_Water'] = False
    UtilitiesDf['Utilities_Gas'] = False
    UtilitiesDf['Utilities_Sewage'] = False
    UtilitiesDf['Utilities_None'] = True
UtilitiesDf = UtilitiesDf.drop('Utilities', axis=1)
df1 = df1.drop('Utilities', axis=1)
df1 = pd.concat([df1.iloc[:, :41], UtilitiesDf, df1.iloc[:, 41:]], axis=1)
#####
LotConfigDf = pd.DataFrame(df1['LotConfig'])
LotConfigDfEncoded = pd.get_dummies(LotConfigDf, columns=['LotConfig'], prefix='LotConfig')
df1 = df1.drop('LotConfig', axis=1)
df1 = pd.concat([df1.iloc[:, :46], LotConfigDfEncoded, df1.iloc[:, 46:]], axis=1)
#####
# Neighborhood'a en son gelicem
#####

In [4]:
df1_1 = df1.iloc[:, :52]
df1_2 = df1.iloc[:, 52:]

df1_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 52 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  2919 non-null   int64  
 1   MSSubClass_20       2919 non-null   bool   
 2   MSSubClass_30       2919 non-null   bool   
 3   MSSubClass_40       2919 non-null   bool   
 4   MSSubClass_45       2919 non-null   bool   
 5   MSSubClass_50       2919 non-null   bool   
 6   MSSubClass_60       2919 non-null   bool   
 7   MSSubClass_70       2919 non-null   bool   
 8   MSSubClass_75       2919 non-null   bool   
 9   MSSubClass_80       2919 non-null   bool   
 10  MSSubClass_85       2919 non-null   bool   
 11  MSSubClass_90       2919 non-null   bool   
 12  MSSubClass_120      2919 non-null   bool   
 13  MSSubClass_150      2919 non-null   bool   
 14  MSSubClass_160      2919 non-null   bool   
 15  MSSubClass_180      2919 non-null   bool   
 16  MSSubC

#

## - Condition1/Condition2 - BldgType - HouseStyle - OverallQual - OverallCond - YearBuilt/YearRemodAdd - RoofStyle/RoofMatl

In [5]:
#####
ConditionDf = df1[['Condition1', 'Condition2']].copy()
conditions = [
    ('Artery', 'Artery', 'AA'), ('Artery', 'Norm', 'AN'), ('Artery', 'PosA', 'APA'),
    ('Feedr', 'Feedr', 'FF'), ('Feedr', 'Norm', 'FN'), ('Feedr', 'RRAe', 'FAe'),
    ('Feedr', 'RRAn', 'FAn'), ('Feedr', 'RRNn', 'FNn'), ('Norm', 'Norm', 'NN'),
    ('PosA', 'Norm', 'PAN'), ('PosN', 'Norm', 'PNN'), ('PosN', 'PosN', 'PNPN'),
    ('RRAe', 'Norm', 'AeN'), ('RRAn', 'Feedr', 'AnF'), ('RRAn', 'Norm', 'AnN'),
    ('RRNe', 'Norm', 'NeN'), ('RRNn', 'Feedr', 'NnF'), ('RRNn', 'Norm', 'NnN')
]
for condition in conditions:
    ConditionDf[condition[2]] = np.where((ConditionDf['Condition1'] == condition[0]) & (ConditionDf['Condition2'] == condition[1]), True, False)
ConditionDfSon = ConditionDf.drop(['Condition1','Condition2'], axis=1)
df1 = df1.drop(['Condition1','Condition2'],axis=1)
df1 = pd.concat([df1.iloc[:, :52], ConditionDfSon, df1.iloc[:, 52:]], axis=1)
#####
BldgDf = df1['BldgType'].copy()
BldgDfEncoded = pd.get_dummies(BldgDf, columns=['BldgType'], prefix='BldgType')
df1 = df1.drop(['BldgType'], axis=1)
df1 = pd.concat([df1.iloc[:, :70], BldgDfEncoded, df1.iloc[:, 70:]], axis=1)
#####
HouseStyleDf = df1['HouseStyle'].copy()
HouseStyleDfEncoded = pd.get_dummies(HouseStyleDf, columns=['HouseStyle'], prefix='HouseStyle')
df1 = df1.drop(['HouseStyle'], axis=1)
df1 = pd.concat([df1.iloc[:, :75], HouseStyleDfEncoded, df1.iloc[:, 75:]], axis=1)
#####
# OverallQual ve OverallCond ordinal olduğu için onlara dokunmayı tercih etmedim.
#####
YearDf = df1[['YearBuilt', 'YearRemodAdd']].copy()
YearDf['RemodelBoolean'] = False
for index, row in YearDf.iterrows():
    if row['YearRemodAdd'] - row['YearBuilt'] > 0:
        YearDf.at[index, 'RemodelBoolean'] = True
    else:
        YearDf.at[index, 'RemodelBoolean'] = False
YearDfSon = YearDf.drop(['YearRemodAdd','YearBuilt'], axis=1)
df1 = df1.drop('YearRemodAdd',axis=1)
df1 = pd.concat([df1.iloc[:, :86], YearDfSon, df1.iloc[:, 86:]], axis=1)
#####
RoofDf = df1[['RoofStyle','RoofMatl']].copy()
roofs = [
    ('Flat', 'CompShg', 'Roof_FCS'), ('Flat', 'Membran', 'Roof_FMembran'), ('Flat', 'Metal', 'Roof_FMetal'),
    ('Flat', 'Tar&Grv', 'Roof_FTG'), ('Gable', 'CompShg', 'Roof_GableC'), ('Gable', 'Roll', 'Roof_GR'),
    ('Gable', 'Tar&Grv', 'Roof_GTG'), ('Gable', 'WdShngl', 'Roof_GW'), ('Gambrel', 'CompShg', 'Roof_GambrelC'),
    ('Hip', 'ClyTile', 'Roof_HCT'), ('Hip', 'CompShg', 'Roof_HCS'), ('Hip', 'WdShake', 'Roof_HWShake'),
    ('Hip', 'WdShngl', 'Roof_HWShngl'), ('Mansard', 'CompShg', 'Roof_MCS'), ('Mansard', 'WdShake', 'Roof_MWS'),
    ('Shed', 'CompShg', 'Roof_SCS'), ('Shed', 'WdShake', 'Roof_SWS')
]
for roof in roofs:
    RoofDf[roof[2]] = np.where((RoofDf['RoofStyle'] == roof[0]) & (RoofDf['RoofMatl'] == roof[1]), True, False)
RoofDfSon = RoofDf.drop(['RoofStyle','RoofMatl'],axis=1)
df1 = df1.drop(['RoofStyle','RoofMatl'],axis=1)
df1 = pd.concat([df1.iloc[:, :87], RoofDfSon, df1.iloc[:, 87:]], axis=1)
#####

In [6]:
df1_1 = df1.iloc[:, :52]
df1_2 = df1.iloc[:, 52:]

df1_2_1 = df1_2.iloc[:, :52]
df1_2_2 = df1_2.iloc[:, 52:]
df1_2_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 52 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   AA                 2919 non-null   bool 
 1   AN                 2919 non-null   bool 
 2   APA                2919 non-null   bool 
 3   FF                 2919 non-null   bool 
 4   FN                 2919 non-null   bool 
 5   FAe                2919 non-null   bool 
 6   FAn                2919 non-null   bool 
 7   FNn                2919 non-null   bool 
 8   NN                 2919 non-null   bool 
 9   PAN                2919 non-null   bool 
 10  PNN                2919 non-null   bool 
 11  PNPN               2919 non-null   bool 
 12  AeN                2919 non-null   bool 
 13  AnF                2919 non-null   bool 
 14  AnN                2919 non-null   bool 
 15  NeN                2919 non-null   bool 
 16  NnF                2919 non-null   bool 
 17  NnN           

#

## - Exterior1st/Exterior2nd

In [7]:
#####
ExteriorDf = df1[['Exterior1st','Exterior2nd']].copy()
ExteriorDf = ExteriorDf.fillna('None')
ExteriorDf['Exteriors']=ExteriorDf[["Exterior1st", "Exterior2nd"]].apply("-".join, axis=1)
ExteriorDfSon = ExteriorDf.drop(['Exterior1st','Exterior2nd'], axis=1)
ExteriorDfSonEncoded = pd.get_dummies(ExteriorDfSon, columns=['Exteriors'], prefix='Exteriors')
df1 = df1.drop(['Exterior1st','Exterior2nd'], axis=1)
df1 = pd.concat([df1.iloc[:, :104], ExteriorDfSonEncoded, df1.iloc[:, 104:]], axis=1)
#####

In [8]:
df1_1 = df1.iloc[:, :52]
df1_2 = df1.iloc[:, 52:]

df1_2_1 = df1_2.iloc[:, :52]
df1_2_2 = df1_2.iloc[:, 52:]

df1_2_2_1 = df1_2_2.iloc[:, :87]
df1_2_2_2 = df1_2_2.iloc[:, 87]
df1_2_2_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 87 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   Exteriors_AsbShng-AsbShng  2919 non-null   bool 
 1   Exteriors_AsbShng-CmentBd  2919 non-null   bool 
 2   Exteriors_AsbShng-Plywood  2919 non-null   bool 
 3   Exteriors_AsbShng-Stucco   2919 non-null   bool 
 4   Exteriors_AsbShng-VinylSd  2919 non-null   bool 
 5   Exteriors_AsbShng-Wd Sdng  2919 non-null   bool 
 6   Exteriors_AsphShn-AsphShn  2919 non-null   bool 
 7   Exteriors_BrkComm-Brk Cmn  2919 non-null   bool 
 8   Exteriors_BrkComm-Stucco   2919 non-null   bool 
 9   Exteriors_BrkComm-Wd Sdng  2919 non-null   bool 
 10  Exteriors_BrkFace-AsbShng  2919 non-null   bool 
 11  Exteriors_BrkFace-BrkFace  2919 non-null   bool 
 12  Exteriors_BrkFace-HdBoard  2919 non-null   bool 
 13  Exteriors_BrkFace-MetalSd  2919 non-null   bool 
 14  Exteriors_BrkFace-Plywoo

#

## - MasVnrType - MasVnrArea - ExterQual - ExterCond - Foundation - BsmtQual - BsmtCond

In [9]:
#####
MasVnrTypeDf = df1['MasVnrType'].copy()
MasVnrTypeDf = MasVnrTypeDf.fillna('None')
MasVnrTypeDfEncoded = pd.get_dummies(MasVnrTypeDf, columns=['MasVnrType'], prefix='MasVnrType')
df1 = df1.drop('MasVnrType', axis=1)
df1 = pd.concat([df1.iloc[:, :191], MasVnrTypeDfEncoded, df1.iloc[:, 191:]], axis=1)
#####
df1['MasVnrArea'].fillna(0, inplace=True)
#####
df1['ExterQual']=df1['ExterQual'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}).astype('int64')
#####
df1['ExterCond']=df1['ExterCond'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}).astype('int64')
#####
FoundationDf = df1[['Foundation']].copy()
FoundationDfEncoded = pd.get_dummies(FoundationDf, columns=['Foundation'], prefix='Foundation')
df1 = df1.drop(['Foundation'], axis=1)
df1 = pd.concat([df1.iloc[:, :198], FoundationDfEncoded, df1.iloc[:, 198:]], axis=1)
#####
df1['BsmtQual'].fillna('NoBasement', inplace=True)
BsmtQualDf = df1['BsmtQual'].copy()
BsmtQualDfEncoded = pd.get_dummies(BsmtQualDf, columns=['BsmtQual'], prefix='BsmtQual')
df1 = df1.drop(['BsmtQual'], axis=1)
df1 = pd.concat([df1.iloc[:, :204], BsmtQualDfEncoded, df1.iloc[:, 204:]], axis=1)
#####
df1['BsmtCond'].fillna('NoBasement', inplace=True)
BsmtCondDf = df1['BsmtCond'].copy()
BsmtCondDfEncoded = pd.get_dummies(BsmtCondDf, columns=['BsmtCond'], prefix='BsmtCond')
df1 = df1.drop(['BsmtCond'], axis=1)
df1 = pd.concat([df1.iloc[:, :209], BsmtCondDfEncoded, df1.iloc[:, 209:]], axis=1)
#####
df1['BsmtExposure'].fillna('NoBasement', inplace=True)
BsmtExposureDf = df1['BsmtExposure'].copy()
BsmtExposureDfEncoded = pd.get_dummies(BsmtExposureDf, columns=['BsmtExposure'], prefix='BsmtExposure')
df1 = df1.drop(['BsmtExposure'], axis=1)
df1 = pd.concat([df1.iloc[:, :214], BsmtExposureDfEncoded, df1.iloc[:, 214:]], axis=1)
#####
df1['BsmtFinType1'].fillna('NoBasement', inplace=True)
BsmtFinType1Df = df1['BsmtFinType1'].copy()
BsmtFinType1DfEncoded = pd.get_dummies(BsmtFinType1Df, columns=['BsmtFinType1'], prefix='BsmtFinType1')
df1 = df1.drop(['BsmtFinType1'], axis=1)
df1 = pd.concat([df1.iloc[:, :219], BsmtFinType1DfEncoded, df1.iloc[:, 219:]], axis=1)
#####
df1['BsmtFinSF1'].fillna(0, inplace=True)
#####
df1['BsmtFinType2'].fillna('NoBasement', inplace=True)
BsmtFinType2Df = df1['BsmtFinType2'].copy()
BsmtFinType2DfEncoded = pd.get_dummies(BsmtFinType2Df, columns=['BsmtFinType2'], prefix='BsmtFinType2')
df1 = df1.drop(['BsmtFinType2'], axis=1)
df1 = pd.concat([df1.iloc[:, :227], BsmtFinType2DfEncoded, df1.iloc[:, 227:]], axis=1)
#####
df1['BsmtFinSF2'].fillna(0, inplace=True)
#####
df1['BsmtUnfSF'].fillna(0, inplace=True)
#####
df1['TotalBsmtSF'].fillna(0, inplace=True)
#####
HeatingDf = df1['Heating'].copy()
HeatingDfEncoded = pd.get_dummies(HeatingDf, columns=['HeatingQC'], prefix='HeatingQc')
df1 = df1.drop(['Heating'], axis=1)
df1 = pd.concat([df1.iloc[:, :237], HeatingDfEncoded, df1.iloc[:, 237:]], axis=1)
#####
df1['HeatingQC']=df1['HeatingQC'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}).astype('int64')
#####
df1['CentralAir']=df1['CentralAir'].replace({'N': False, 'Y': True}).astype(bool)
#####
df1['Electrical'].fillna('SBrkr', inplace=True)
ElectricalDf = df1[['Electrical']].copy()
ElectricalDfEncoded = pd.get_dummies(ElectricalDf, columns=['Electrical'], prefix='Electrical')
df1 = df1.drop(['Electrical'], axis=1)
df1 = pd.concat([df1.iloc[:, :245], ElectricalDfEncoded, df1.iloc[:, 245:]], axis=1)
#####
df1['BsmtFullBath'].fillna(0, inplace=True)
#####
df1['BsmtHalfBath'].fillna(0, inplace=True)
#####
df1['KitchenQual'].fillna('TA', inplace=True)
df1['KitchenQual']=df1['KitchenQual'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}).astype('int64')
#####
df1['Functional'].fillna('Typ', inplace=True)
FunctionalDf = df1[['Functional']].copy()
FunctionalDfEncoded = pd.get_dummies(FunctionalDf, columns=['Functional'], prefix='Functional')
df1 = df1.drop(['Functional'], axis=1)
df1 = pd.concat([df1.iloc[:, :262], FunctionalDfEncoded, df1.iloc[:, 262:]], axis=1)
#####

In [10]:
df1_1 = df1.iloc[:, :52]
df1_2 = df1.iloc[:, 52:]

df1_2_1 = df1_2.iloc[:, :52]
df1_2_2 = df1_2.iloc[:, 52:]

df1_2_2_1 = df1_2_2.iloc[:, :87]
df1_2_2_2 = df1_2_2.iloc[:, 87:]

df1_2_2_2_1 = df1_2_2_2.iloc[:, :78]
df1_2_2_2_2 = df1_2_2_2.iloc[:, 78:]
df1_2_2_2_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 78 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   MasVnrType_BrkCmn        2919 non-null   bool   
 1   MasVnrType_BrkFace       2919 non-null   bool   
 2   MasVnrType_None          2919 non-null   bool   
 3   MasVnrType_Stone         2919 non-null   bool   
 4   MasVnrArea               2919 non-null   float64
 5   ExterQual                2919 non-null   int64  
 6   ExterCond                2919 non-null   int64  
 7   Foundation_BrkTil        2919 non-null   bool   
 8   Foundation_CBlock        2919 non-null   bool   
 9   Foundation_PConc         2919 non-null   bool   
 10  Foundation_Slab          2919 non-null   bool   
 11  Foundation_Stone         2919 non-null   bool   
 12  Foundation_Wood          2919 non-null   bool   
 13  BsmtQual_Ex              2919 non-null   bool   
 14  BsmtQual_Fa             

#

## - FirePlacesQu - GarageType - GarageYrBlt - GarageFinish - GarageCars - GarageArea - GarageQual - GarageCond - PavedDrive - PoolQC - Fence - MiscFeature - SaleType - SaleCondition

In [11]:
#####
df1['FireplaceQu'].fillna('None', inplace=True)
FireplaceQuDf = df1[['FireplaceQu']].copy()
FireplaceQuDfEncoded = pd.get_dummies(FireplaceQuDf, columns=['FireplaceQu'], prefix='FireplaceQu')
df1 = df1.drop(['FireplaceQu'], axis=1)
df1 = pd.concat([df1.iloc[:, :270], FireplaceQuDfEncoded, df1.iloc[:, 270:]], axis=1)
#####
df1.loc[df1['Id'].isin([2577]), 'GarageType'] = pd.NA
df1['GarageType'].fillna('NoGarage', inplace=True)
GarageTypeDf = df1['GarageType'].copy()
GarageTypeDfEncoded = pd.get_dummies(GarageTypeDf, columns=['GarageType'], prefix='GarageType')
df1 = df1.drop(['GarageType'], axis=1)
df1 = pd.concat([df1.iloc[:, :276], GarageTypeDfEncoded, df1.iloc[:, 276:]], axis=1)
#####
df1.loc[df1['Id'].isin([2127]), 'GarageYrBlt'] = df1['YearBuilt']
#####
df1.loc[df1['Id'].isin([2127]), 'GarageFinish'] = 'Unf'
df1['GarageFinish'].fillna('None', inplace=True)
GarageFinishDf = df1[['GarageFinish']].copy()
GarageFinishDfEncoded = pd.get_dummies(GarageFinishDf, columns=['GarageFinish'], prefix='GarageFinish')
df1 = df1.drop(['GarageFinish'], axis=1)
df1 = pd.concat([df1.iloc[:, :284], GarageFinishDfEncoded, df1.iloc[:, 284:]], axis=1)
#####
df1['GarageCars'].fillna(0 , inplace=True)
#####
df1['GarageArea'].fillna(0 , inplace=True)
#####
df1.loc[df1['Id'].isin([2127]), 'GarageQual'] = 'TA'
df1['GarageQual'].fillna('None', inplace=True)
GarageQualDf = df1[['GarageQual']].copy()
GarageQualDfEncoded = pd.get_dummies(GarageQualDf, columns=['GarageQual'], prefix='GarageQual')
df1 = df1.drop(['GarageQual'], axis=1)
df1 = pd.concat([df1.iloc[:, :290], GarageQualDfEncoded, df1.iloc[:, 290:]], axis=1)
#####
df1.loc[df1['Id'].isin([2127]), 'GarageCond'] = 'TA'
df1['GarageCond'].fillna('None', inplace=True)
GarageCondDf = df1[['GarageCond']].copy()
GarageCondDfEncoded = pd.get_dummies(GarageCondDf, columns=['GarageCond'], prefix='GarageCond')
df1 = df1.drop(['GarageCond'], axis=1)
df1 = pd.concat([df1.iloc[:, :296], GarageCondDfEncoded, df1.iloc[:, 296:]], axis=1)
#####
PavedDriveDf = df1[['PavedDrive']].copy()
PavedDriveDfEncoded = pd.get_dummies(PavedDriveDf, columns=['PavedDrive'], prefix='PavedDrive')
df1 = df1.drop(['PavedDrive'], axis=1)
df1 = pd.concat([df1.iloc[:, :302], PavedDriveDfEncoded, df1.iloc[:, 302:]], axis=1)
#####
df1['PoolQC'].fillna('None', inplace=True)
PoolQCDf = df1[['PoolQC']].copy()
PoolQCDfEncoded = pd.get_dummies(PoolQCDf, columns=['PoolQC'], prefix='PoolQC')
df1 = df1.drop(['PoolQC'], axis=1)
df1 = pd.concat([df1.iloc[:, :311], PoolQCDfEncoded, df1.iloc[:, 311:]], axis=1)
#####
df1['Fence'].fillna('None', inplace=True)
FenceDf = df1[['Fence']].copy()
FenceDfEncoded = pd.get_dummies(FenceDf, columns=['Fence'], prefix='Fence')
df1 = df1.drop(['Fence'], axis=1)
df1 = pd.concat([df1.iloc[:, :315], FenceDfEncoded, df1.iloc[:, 315:]], axis=1)
#####
df1['MiscFeature'].fillna('None', inplace=True)
MiscFeatureDf = df1[['MiscFeature']].copy()
MiscFeatureDfEncoded = pd.get_dummies(MiscFeatureDf, columns=['MiscFeature'], prefix='MiscFeature')
df1 = df1.drop(['MiscFeature'], axis=1)
df1 = pd.concat([df1.iloc[:, :320], MiscFeatureDfEncoded, df1.iloc[:, 320:]], axis=1)
#####
df1['SaleType'].fillna('WD', inplace=True)
SaleTypeDf = df1[['SaleType']].copy()
SaleTypeDfEncoded = pd.get_dummies(SaleTypeDf, columns=['SaleType'], prefix='SaleType')
df1 = df1.drop(['SaleType'], axis=1)
df1 = pd.concat([df1.iloc[:, :328], SaleTypeDfEncoded, df1.iloc[:, 328:]], axis=1)
#####
SaleConditionDf = df1[['SaleCondition']].copy()
SaleConditionDfEncoded = pd.get_dummies(SaleConditionDf, columns=['SaleCondition'], prefix='SaleCondition')
df1 = df1.drop(['SaleCondition'], axis=1)
df1 = pd.concat([df1.iloc[:, :337], SaleConditionDfEncoded, df1.iloc[:, 337:]], axis=1)

In [12]:
df1_1 = df1.iloc[:, :52]
df1_2 = df1.iloc[:, 52:]

df1_2_1 = df1_2.iloc[:, :52]
df1_2_2 = df1_2.iloc[:, 52:]

df1_2_2_1 = df1_2_2.iloc[:, :87]
df1_2_2_2 = df1_2_2.iloc[:, 87:]

df1_2_2_2_1 = df1_2_2_2.iloc[:, :78]
df1_2_2_2_2 = df1_2_2_2.iloc[:, 78:]
df1_2_2_2_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 74 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Fireplaces             2919 non-null   int64  
 1   FireplaceQu_Ex         2919 non-null   bool   
 2   FireplaceQu_Fa         2919 non-null   bool   
 3   FireplaceQu_Gd         2919 non-null   bool   
 4   FireplaceQu_None       2919 non-null   bool   
 5   FireplaceQu_Po         2919 non-null   bool   
 6   FireplaceQu_TA         2919 non-null   bool   
 7   GarageType_2Types      2919 non-null   bool   
 8   GarageType_Attchd      2919 non-null   bool   
 9   GarageType_Basment     2919 non-null   bool   
 10  GarageType_BuiltIn     2919 non-null   bool   
 11  GarageType_CarPort     2919 non-null   bool   
 12  GarageType_Detchd      2919 non-null   bool   
 13  GarageType_NoGarage    2919 non-null   bool   
 14  GarageYrBlt            2761 non-null   float64
 15  Gara

#

## DataFrame'lerin birleştirilmesi

In [13]:
son_df = pd.concat([df1_1, df1_2_1, df1_2_2_1, df1_2_2_2_1, df1_2_2_2_2], axis=1)
son_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Columns: 343 entries, Id to SaleCondition_Partial
dtypes: bool(303), float64(11), int64(28), object(1)
memory usage: 1.7+ MB


In [14]:
for column in son_df.columns:
    nan_count = son_df[column].isnull().sum()
    print(f"{column}: {nan_count} NaN")

Id: 0 NaN
MSSubClass_20: 0 NaN
MSSubClass_30: 0 NaN
MSSubClass_40: 0 NaN
MSSubClass_45: 0 NaN
MSSubClass_50: 0 NaN
MSSubClass_60: 0 NaN
MSSubClass_70: 0 NaN
MSSubClass_75: 0 NaN
MSSubClass_80: 0 NaN
MSSubClass_85: 0 NaN
MSSubClass_90: 0 NaN
MSSubClass_120: 0 NaN
MSSubClass_150: 0 NaN
MSSubClass_160: 0 NaN
MSSubClass_180: 0 NaN
MSSubClass_190: 0 NaN
MSZoning_C (all): 0 NaN
MSZoning_FV: 0 NaN
MSZoning_RH: 0 NaN
MSZoning_RL: 0 NaN
MSZoning_RM: 0 NaN
LotFrontage: 486 NaN
LotArea: 0 NaN
StreetPave: 0 NaN
Alley_Grvl: 0 NaN
Alley_None: 0 NaN
Alley_Pave: 0 NaN
LotShape: 0 NaN
BnkGtl: 0 NaN
BnkMod: 0 NaN
BnkSev: 0 NaN
HLSGtl: 0 NaN
HLSMod: 0 NaN
HLSSev: 0 NaN
LowGtl: 0 NaN
LowMod: 0 NaN
LowSev: 0 NaN
LvlMod: 0 NaN
LvlSev: 0 NaN
GtlLvl: 0 NaN
Utilities_Electric: 0 NaN
Utilities_Water: 0 NaN
Utilities_Gas: 0 NaN
Utilities_Sewage: 0 NaN
Utilities_None: 0 NaN
LotConfig_Corner: 0 NaN
LotConfig_CulDSac: 0 NaN
LotConfig_FR2: 0 NaN
LotConfig_FR3: 0 NaN
LotConfig_Inside: 0 NaN
Neighborhood: 0 NaN
AA: 0 

#